In [0]:
!git clone https://github.com/salesforce/cove.git # use ssh: git@github.com:salesforce/cove.git
!cd cove
!pip install -r cove/requirements.txt
!python cove/setup.py develop
!pip install transformers
# On CPU

fatal: destination path 'cove' already exists and is not an empty directory.
  Cloning git://github.com/jekbradbury/revtok.git to /tmp/pip-req-build-drse4k42
  Running command git clone -q git://github.com/jekbradbury/revtok.git /tmp/pip-req-build-drse4k42
  Cloning https://github.com/pytorch/text.git to /tmp/pip-req-build-rvci_4h4
  Running command git clone -q https://github.com/pytorch/text.git /tmp/pip-req-build-rvci_4h4
  Created wheel for revtok: filename=revtok-0.0.3-cp36-none-any.whl size=5183 sha256=aade184c4b906a09316a26d2dcccac7db1b0fa59aa56a78d15d0ae23ea41c76d
  Stored in directory: /tmp/pip-ephem-wheel-cache-9j_9z_65/wheels/7a/ea/8a/6728347664cd622aaf563f3a46e68338f671f30c1b642006f2
  Created wheel for torchtext: filename=torchtext-0.5.1-cp36-none-any.whl size=64354 sha256=4f7b075aff2150cd3c4a5d39a5780313bbd5673bee625d96d648c193f5faa185
  Stored in directory: /tmp/pip-ephem-wheel-cache-9j_9z_65/wheels/09/f9/ed/0e37c2b62298f9b1225412431e805e89206cbde3a0bac44327
Successfully

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from csv import reader
import csv
import re
import pathlib
import sys
from torchtext import data
from torchtext import datasets
from torchtext.vocab import GloVe
from cove import cove
from transformers import BertTokenizer, AutoTokenizer, BertModel, DistilBertModel
from sklearn.metrics import f1_score

In [0]:
!tar -xvf /content/jessi.tar 

jessi_A_fold_1_epoch_10.pt


In [0]:
# Bert Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

MAX_LEN_GC = 350
MAX_LEN_B = 25

def tokenizerfnc(str):
    return tokenizer.encode(str,max_length=MAX_LEN_B, pad_to_max_length=True, add_special_tokens=True)

In [0]:
class CNN_GC(nn.Module):

    def __init__(self,out_dim=200,gc_dim=900,max_len=MAX_LEN_GC, dropout=0.5):
        super(CNN_GC, self).__init__()
        
        """cove_model = CoVeEmbeddings(
            word_embeddings_dir='../model/text/stanford/glove/', 
            tokenizer=tokenizer,
            max_sequence_length=max_len, verbose=20)
        glove_model = GloVeEmbeddings()
        glove_model.load_model(dest_dir='../model/text/stanford/glove/', process=False)"""
        
        self.gc_dim = gc_dim
        self.out_dim = out_dim
        self.max_len = max_len
        self.dropout = dropout

        self.conv_3 = nn.Conv1d(gc_dim, out_dim, 3, stride=1, padding=1)
        self.conv_5 = nn.Conv1d(gc_dim, out_dim, 5, stride=1, padding=2)
        self.conv_7 = nn.Conv1d(gc_dim, out_dim, 7, stride=1, padding=3)
        self.attn = nn.Linear(3*out_dim*max_len, max_len)

    def forward(self,x):
        #cove_embed = cove_model.encode(x)
        #tokens = [sentence.split(" ") for sentence in x]
        #glove_embed = glove_model.encode(tokens)
        #x = torch.cat([cove_embed,glove_embed], 2)
        
        conv_3 = F.relu(self.conv_3(x))
        conv_5 = F.relu(self.conv_5(x))
        conv_7 = F.relu(self.conv_7(x))
        x = torch.cat([conv_3,conv_5,conv_7], 1)
        #print(x.shape)
        
        non_linear_x = F.relu(x.view(-1, 600*self.max_len))
        #print(non_linear_x.shape)
        attn_weights = F.softmax(self.attn(non_linear_x), dim=1)
        #print(attn_weights.shape)
        #attn_applied = torch.bmm(attn_weights.unsqueeze(1), x)
        #attn_applied = attn_weights*x
        attn_applied = torch.zeros(x.shape[0], x.shape[1], x.shape[2]).to(device)
        for i in range(x.shape[0]):
            attn_applied[i,:,:] = x[i,:,:]*attn_weights[i]
        #print("hello")
        #print(attn_applied.shape)
        
        x = attn_applied.sum(dim=2)
        del attn_applied
        return x
    
class CNN_BERT(nn.Module):

    def __init__(self, out_dim=100, embed_dim=768, max_len=MAX_LEN_B, dropout=0.5):
        super(CNN_BERT, self).__init__()
        
        self.embed_dim = embed_dim
        self.out_dim = out_dim # not 100?
        self.max_len = max_len # ?
        self.dropout = dropout

        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.conv_3 = nn.Conv1d(1, out_dim, embed_dim * 3, stride=embed_dim)
        self.conv_4 = nn.Conv1d(1, out_dim, embed_dim * 4, stride=embed_dim)
        self.conv_5 = nn.Conv1d(1, out_dim, embed_dim * 5, stride=embed_dim)

    def get_conv_out(self,conv,x, num):
        return F.max_pool1d(F.relu(conv(x)), 
                            self.max_len - num).view(-1, self.out_dim)

    def forward(self,x, attn_masks):
        x, _ = self.bert_layer(x, attention_mask = attn_masks)
        #print('bert')
        #print(x)
        x = x[:, 1:].view(-1, 1, self.embed_dim * (self.max_len-1))
        conv_3 = self.get_conv_out(self.conv_3, x, 3)
        conv_4 = self.get_conv_out(self.conv_4, x, 4)
        conv_5 = self.get_conv_out(self.conv_5, x, 5)
        x = torch.cat([conv_3, conv_4, conv_5], 1)
        #x = F.dropout(x, p=0.2)
        return x

In [0]:
########################
# INITIALIZATION

def init_weights(m):
    if type(m) == nn.Conv1d:
        torch.nn.init.xavier_normal_(m.weight.data)
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)


def init_weights_BERT(m):
    if type(m) == nn.Conv1d:
        torch.nn.init.xavier_normal_(m.weight.data)



######################################################################
#
#   Architecture is as follows
#   1. Bert word encoding + CNN with max pooling for sent.embed
#   2. Glove & CoVe word Encoding  + CNN with attention for sent.embed
#   3. (1) and (2) concated and fed to MLP for classification (1-sugg)

class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.l0 = nn.Linear(900,300)
        self.l1 = nn.Linear(300,300)
        self.l3 = nn.Linear(300,2)
        self.act1 = nn.ReLU()
        self.act2 = nn.ReLU()

    def forward(self,x):

        x = (self.l0(x))
        x = self.act1(x)
        x = (self.l1(x))
        x = self.act2(x)
        x = (self.l3(x))
        return x


class SuggestionClassifier(nn.Module):
    def __init__(self):

        super(SuggestionClassifier,self).__init__()
        self.CNN_b = CNN_BERT()
        self.CNN_b.apply(init_weights_BERT)
        self.CNN_gc = CNN_GC()
        self.CNN_gc.apply(init_weights)
        self.MLP = MLP()
        self.MLP.apply(init_weights)


    def forward(self,x, y, attn_masks):
        #print("=========================")
        bert_x = self.CNN_b(x, attn_masks)
        #print("Total Model", bert_x.shape)
        gc_x = self.CNN_gc(y)
        #print("Total Model", gc_x.shape)
        out = torch.cat((bert_x,gc_x), 1) # TODO : ensure dimensions align (1x300, 1x600)
        #print("Total", out.shape)
        out = self.MLP(out) # Dim : 1x900
        #print("Total", out.shape)
        return out

In [0]:
#########################
# HYPER PARAMETER TUNING

NUM_EPOCHS = 10
#WEIGHT_DECAY = 3
BATCH_SIZE = 32
NFOLDS = 10
NUM_WORKERS = 16
LEARNING_RATE = 2e-5 # TODO : Experiment, as LR not given

########################
# MODEL CREATION


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
#############################
# HELPER FUNCTIONS

def train(model, train_loader, valid_loader, eval_label_loader, test_loader, foldId, numEpochs,
           outputs_cove_with_glove,  outputs_cove_with_glove_test, outputs_cove_with_glove_eval):
    '''
    Args:

    Ret: 
    '''
    f = open(ROOT + "log_{}.txt".format(foldId+1),"w+")
    model.train()
    val_loss_min = None
    val_loss_min_delta = 0.005
    val_patience = 5
    val_loss_counter = 0
        
    for epoch in range(numEpochs):
        
        avg_loss = 0.0
        model.train()
        for batch_num, batch in enumerate(train_loader):
            glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
            target = torch.zeros(glove_then_last_layer_cove.shape[0], MAX_LEN_GC, 900)
            max_sentence_len_in_batch = max(batch.sentence[1].tolist())
            target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
            glove_then_last_layer_cove = target.permute(0,2,1)
            
            bert_enc = batch.bert_enc.to(device)
            attn_masks = (bert_enc != 0).long()
            glove_cove_enc = glove_then_last_layer_cove.to(device)
            
            optimizer.zero_grad()
            outputs = model(bert_enc, glove_cove_enc, attn_masks)
            _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
            """
            if batch_num % 50 == 0:
              print(outputs)
              print(pred_labels)
            """
            #print(outputs.shape)
            #print(outputs)
            labels = batch.label.long()
            labels = labels.to(device)
            loss = criterion(outputs, labels)
            """
            if batch_num % 50 == 0:
              print(labels)
              print(loss)
            """
            #print(loss)
            
            loss.backward()
            optimizer.step()
            """
            for n,p in model.named_parameters():
                if p.requires_grad and n.startswith('MLP.l0'):
                  print(n, p.data)
            """
            #print("Optimizer Complete")

            avg_loss += loss.item()
            #print(avg_loss)

            if batch_num % 50 == 49:
                print('Fold: {}\t Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(foldId+1, 
                                                                                 epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0

            torch.cuda.empty_cache()

            del bert_enc
            del glove_cove_enc
            del labels
            

        model_name = "jessi_A_fold_"+str(foldId+1)+"_epoch_"+str(epoch+1)+".pt"
        
        print('Epoch'+str(epoch))
        val_loss, val_sc, val_acc = validate(model, valid_loader, outputs_cove_with_glove_valid)
        eval_loss, eval_sc, eval_acc = validate(model, eval_label_loader, outputs_cove_with_glove_eval)
        test_loss, test_sc,test_acc = validate(model, test_loader, outputs_cove_with_glove_test)
        print('Epoch{:.4f}\tVal Loss: {:.4f}\tVal Sc: {:.4f}\tVal Accuracy: {:.4f}'.format(
             epoch, val_loss, val_sc, val_acc))
        print('Epoch{:.4f}\tEVal Loss: {:.4f}\tEVal Sc: {:.4f}\tEVal Accuracy: {:.4f}'.format(
             epoch, eval_loss, eval_sc, eval_acc))
        print('Epoch{:.4f}\tTest Loss: {:.4f}\tTest sc: {:.4f}\tTest Accuracy: {:.4f}'.format(
            epoch, test_loss, test_sc,test_acc))
        f.write('Epoch{:.4f}\tVal Loss: {:.4f}\tVal Sc: {:.4f}\tVal Accuracy: {:.4f}\n'.format(
             epoch, val_loss, val_sc, val_acc))
        f.write('Epoch{:.4f}\tEVal Loss: {:.4f}\tEVal Sc: {:.4f}\tEVal Accuracy: {:.4f}\n'.format(
             epoch, eval_loss, eval_sc, eval_acc))
        f.write('Epoch{:.4f}\tTest Loss: {:.4f}\tTest sc: {:.4f}\tTest Accuracy: {:.4f}\n'.format(
            epoch, test_loss, test_sc,test_acc))
        torch.save(model.state_dict(), ROOT+model_name)

    f.close()
    return test_acc, model_name #test_acc


def validate(model, valid_loader, outputs_cove_with_glove):
    '''
    Args:

    Ret:
    '''

    model.eval()
    test_loss = []
    all_predictions = []
    all_labels = []
    accuracy = 0
    total = 0

    for batch_num, batch in enumerate(valid_loader):
        
        glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
        target = torch.zeros(glove_then_last_layer_cove.shape[0], MAX_LEN_GC, 900)
        max_sentence_len_in_batch = max(batch.sentence[1].tolist())
        target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
        glove_then_last_layer_cove = target.permute(0,2,1)

        bert_enc = batch.bert_enc.to(device)
        attn_masks = (bert_enc != 0).long()
        glove_cove_enc = glove_then_last_layer_cove.to(device)
        #print(glove_cove_enc)
        outputs = model(bert_enc, glove_cove_enc, attn_masks)

        labels = batch.label.long()
        labels = labels.to(device)
    
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        loss = criterion(outputs, labels).item()
        accuracy += torch.sum(torch.eq(pred_labels, labels.to(device))).item()
        total += len(labels)

        test_loss.extend([loss]*len(labels))
        
        all_labels.extend(list(np.array(labels.cpu())))
        predictions = pred_labels.cpu()
        all_predictions.extend(list(np.array(predictions)))
        

        #torch.cuda.empty_cache()
        del bert_enc
        del glove_cove_enc
        del labels
    
        

    score = f1_score(all_labels, all_predictions)
    print(all_labels)
    print(all_predictions)
    print(accuracy)
    print(total)
    #return 0, score
    return np.mean(test_loss),score, accuracy/total



In [0]:
########################
# DATA LOADING

ROOT = "/content/drive/My Drive/NN for NLP/"

VALID_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
VALID_LABEL = data.Field(sequential=False, use_vocab=False)
VALID_TEXT_BERT = data.Field(
    use_vocab=False,
    batch_first=True,
    pad_token=tokenizer.pad_token_id,
    tokenize=tokenizerfnc
)

# valid_dataset = SuggestionDataset(load_data(filename="SubtaskA_Trial_Test_Labeled.csv"))
valid_path = ROOT + "SubtaskA_Trial_Test_Labeled.csv"
valid_dataset = data.TabularDataset(
            path=valid_path, format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',VALID_TEXT),('bert_enc',VALID_TEXT_BERT)],
                    'label':('label',VALID_LABEL)
                    })



VALID_TEXT.build_vocab(valid_dataset, vectors=GloVe(name='840B', dim=300, cache='/content/drive/My Drive/nn4nlpdata'))
outputs_cove_with_glove_valid = cove.MTLSTM(n_vocab=len(VALID_TEXT.vocab), vectors=VALID_TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

valid_iter = data.Iterator(
        (valid_dataset),
        batch_size=BATCH_SIZE)

eval_path =  ROOT + 'SubtaskA_EvaluationData_labeled.csv' #'test_{}.csv'.format(i)
    # test loader

TEST_TEXT1 = data.Field(lower=True, include_lengths=True, batch_first=True)
TEST_LABEL1 = data.Field(sequential=False, use_vocab=False)
TEST_TEXT_BERT1 = data.Field(
        use_vocab=False,
        batch_first=True,
        pad_token=tokenizer.pad_token_id,
        tokenize=tokenizerfnc
    )

test_dataset1 = data.TabularDataset(
            path=eval_path, format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEST_TEXT1),('bert_enc',TEST_TEXT_BERT1)],
                    'label':('label',TEST_LABEL1)
                    })

TEST_TEXT1.build_vocab(test_dataset1, vectors=GloVe(name='840B', dim=300, cache='/content/drive/My Drive/nn4nlpdata'))
outputs_cove_with_glove_eval = cove.MTLSTM(n_vocab=len(TEST_TEXT1.vocab), vectors=TEST_TEXT1.vocab.vectors, 
                                          residual_embeddings=True, model_cache=ROOT)

eval_label_iter = data.Iterator(
        (test_dataset1),
        batch_size=BATCH_SIZE)

In [0]:
# data = load_data()
# data_folds = create_folds(data)
model_perfs = []
model_names = []

train_path = ROOT + "train_{}.csv"
test_path = ROOT + "test_{}.csv"
#2
for i in [1,2,3,4,6,7,9]:
    
    print("FOLDID: ", i)
    print("FOLD: ", i+1)
    
    model = SuggestionClassifier()
    model.train()
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    LABEL = data.Field(sequential=False, use_vocab=False)
    TEXT_BERT = data.Field(
        use_vocab=False,
        batch_first=True,
        pad_token=tokenizer.pad_token_id,
        tokenize=tokenizerfnc
    )

    # train loader
    train_dataset = data.TabularDataset(
            path=train_path.format(i), format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
                    'label':('label',LABEL)
                    })

    TEXT.build_vocab(train_dataset, vectors=GloVe(name='840B', dim=300, cache=ROOT))
    outputs_cove_with_glove = cove.MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, 
                                     residual_embeddings=True, model_cache=ROOT + ".embeddings")

    train_iter = data.Iterator(
        (train_dataset),
        batch_size=BATCH_SIZE)
    
    print("Created Train Iterator")
    
    TEST_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    TEST_LABEL = data.Field(sequential=False, use_vocab=False)
    TEST_TEXT_BERT = data.Field(
        use_vocab=False,
        batch_first=True,
        pad_token=tokenizer.pad_token_id,
        tokenize=tokenizerfnc
    )
    
    # test loader
    test_dataset = data.TabularDataset(
            path=test_path.format(i), format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEST_TEXT),('bert_enc',TEST_TEXT_BERT)],
                    'label':('label',TEST_LABEL)
                    })

    TEST_TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache=ROOT))
    outputs_cove_with_glove_test = cove.MTLSTM(n_vocab=len(TEST_TEXT.vocab), vectors=TEST_TEXT.vocab.vectors, 
                                          residual_embeddings=True, model_cache=ROOT + ".embeddings")

    test_iter = data.Iterator(
        (test_dataset),
        batch_size=BATCH_SIZE)
    
    print("Created Test Iterator")
    
    test_loss, model_name = train(model, train_iter, valid_iter, eval_label_iter ,test_iter, i, NUM_EPOCHS, 
                                  outputs_cove_with_glove, outputs_cove_with_glove_test, outputs_cove_with_glove_eval)

    model_perfs.append(test_loss)
    model_names.append(model_name)
    del model
    del criterion
    del optimizer
    
print(model_perfs)
print(model_names)
best_three_model_idx = np.flip(np.argsort(model_perfs))[0:3]
models = []
for idx in best_three_model_idx:
    # load the model
    #temp_model = SuggestionClassifier()
    temp_model = SuggestionClassifier()
    temp_model.load_state_dict(torch.load(model_names[idx]))
    models.append(temp_model)
# testing
print(len(models))

FOLDID:  1
FOLD:  2
Created Train Iterator
Created Test Iterator
Fold: 2	 Epoch: 1	Batch: 50	Avg-Loss: 0.5261
Fold: 2	 Epoch: 1	Batch: 100	Avg-Loss: 0.2906
Fold: 2	 Epoch: 1	Batch: 150	Avg-Loss: 0.2578
Fold: 2	 Epoch: 1	Batch: 200	Avg-Loss: 0.2634
Epoch0
[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 

In [0]:
torch.cuda.get_device_properties(device)

_CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)

In [0]:
!ls "/content/drive/My Drive/nnnlpmodels" .

.:
cove	       SubtaskA_EvaluationData_labeled.csv  test_7.csv	 train_6.csv
cove.egg-info  SubtaskA_Trial_Test_Labeled.csv	    test_8.csv	 train_7.csv
drive	       test_5.csv			    test_9.csv	 train_8.csv
sample_data    test_6.csv			    train_5.csv  train_9.csv

'/content/drive/My Drive/nnnlpmodels':


In [0]:
def loop(i):
  model = SuggestionClassifier()
  model.eval()
  
  model.load_state_dict(torch.load('jessi_A_fold_{}_epoch_4.pt'.format(i+1)))
  model.to(device)
  valid_path='/content/SubtaskA_EvaluationData_labeled.csv' #'test_{}.csv'.format(i)
    # test loader

  TEST_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
  TEST_LABEL = data.Field(sequential=False, use_vocab=False)
  TEST_TEXT_BERT = data.Field(
        use_vocab=False,
        batch_first=True,
        pad_token=tokenizer.pad_token_id,
        tokenize=tokenizerfnc
    )
  test_dataset = data.TabularDataset(
            path=valid_path, format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEST_TEXT),('bert_enc',TEST_TEXT_BERT)],
                    'label':('label',TEST_LABEL)
                    })
  TEST_TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache='/content/drive/My Drive/nn4nlpdata'))
  outputs_cove_with_glove_test = cove.MTLSTM(n_vocab=len(TEST_TEXT.vocab), vectors=TEST_TEXT.vocab.vectors, 
                                          residual_embeddings=True, model_cache='/content/drive/My Drive/nn4nlpdata')
  test_iter = data.Iterator(
        (test_dataset),
        batch_size=BATCH_SIZE)
  score, acc = validate(model, test_iter, outputs_cove_with_glove_test)
  print('Epoch'+str(i),score,acc)
  del model

In [0]:

for i in range(4,5):
  loop(i)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

In [0]:
models = []
for item in [(5,6),(6,8),(9,4),(1,7)]:
  model = SuggestionClassifier()
  model.load_state_dict(torch.load('jessi_A_fold_{}_epoch_{}.pt'.format(item[0],item[1])))
  models.append(model)


In [0]:
!cp "/content/drive/My Drive/nn4nlp_best_models/jessi_A_fold_6_epoch_8.pt" .

In [0]:
def test(models, test_loader, outputs_cove_with_glove, id_map=0):
    '''
    Args:

    Ret:
    '''

    test_loss = []
    accuracy = 0
    total = 0
    
    all_labels = []
    all_predictions = []
    outcomes = []
    
    for batch_num, batch in enumerate(test_loader):

        glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
        target = torch.zeros(glove_then_last_layer_cove.shape[0], MAX_LEN_GC, 900)
        max_sentence_len_in_batch = max(batch.sentence[1].tolist())
        target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
        glove_then_last_layer_cove = target.permute(0,2,1)

        bert_enc = batch.bert_enc.to(device)
        glove_cove_enc = glove_then_last_layer_cove.to(device)
        
        predictions = torch.tensor(np.zeros(len(batch.label)))

        for model in models:
            model.eval()
            model.to(device)
            outputs = model(bert_enc, glove_cove_enc)
            
            _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
            # print(np.array(pred_labels.view(-1).cpu()))
            predictions += pred_labels.view(-1).cpu()
            model.train()
            del model

        predictions = torch.round(predictions/len(models))
        
        labels = batch.label.long()
        
        accuracy += torch.sum(torch.eq(predictions, labels))
        total += len(labels)
        
        all_labels.extend(list(np.array(labels)))
        
        predictions = np.array(predictions)
        all_predictions.extend(list(predictions))
        
        outcomes += [ ['ID','SENT',predictions[i]] for i in range(len(predictions)) ]
        
        del bert_enc
        del glove_cove_enc
        del labels
    
    f = open("result.csv", "w", newline="")
    writer = csv.writer(f)
    writer.writerows(outcomes)
    f.close()
    
    score = f1_score(all_labels, all_predictions)
    # print(all_labels, all_predictions)
    print("F1 Score:", str(score))
    
    #return np.mean(test_loss), accuracy/total
    return accuracy/total, score


TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)
TEXT_BERT = data.Field(
    use_vocab=False,
    batch_first=True,
    pad_token=tokenizer.pad_token_id,
    tokenize=tokenizerfnc
)

test_path = "/content/SubtaskA_EvaluationData_labeled.csv"
test_dataset = data.TabularDataset(
            path=test_path, format='csv',
            skip_header = False,
            fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
                    'label':('label',LABEL)
                    })


TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache='/content/drive/My Drive/nn4nlpdata'))
outputs_cove_with_glove = cove.MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')


test_iter = data.Iterator(
        (test_dataset),
        batch_size=16)

acc,score = test(models, test_iter, outputs_cove_with_glove)

F1 Score: 0.5829145728643216


In [0]:
print(acc,score)

tensor(0) 0.6666666666666666


In [0]:
# i = 1
# train_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\train_{}.csv"
# test_path = "C:\\Users\\thank\\Downloads\\CMU MCDS\\11747\\project\suggestionMining\\data\\Subtask-A\\test_{}.csv"

# TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
# LABEL = data.Field(sequential=False, use_vocab=False)
# TEXT_BERT = data.Field(
#     use_vocab=False,
#     batch_first=True,
#     pad_token=tokenizer.pad_token_id,
#     tokenize=tokenizerfnc
# )

# # train loader
# train_dataset = data.TabularDataset(
#         path=train_path.format(i), format='csv',
#         skip_header = False,
#         fields={'sentence':[('sentence',TEXT),('bert_enc',TEXT_BERT)],
#                 'label':('label',LABEL)
#                 })

# TEXT.build_vocab(train_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
# outputs_cove_with_glove = MTLSTM(n_vocab=len(TEXT.vocab), vectors=TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

# train_iter = data.Iterator(
#     (train_dataset),
#     batch_size=5)


# print("Created Train Iterator")

# TEST_TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
# TEST_LABEL = data.Field(sequential=False, use_vocab=False)
# TEST_TEXT_BERT = data.Field(
#     use_vocab=False,
#     batch_first=True,
#     pad_token=tokenizer.pad_token_id,
#     tokenize=tokenizerfnc
# )

# # test loader
# test_dataset = data.TabularDataset(
#         path=test_path.format(i), format='csv',
#         skip_header = False,
#         fields={'sentence':[('sentence',TEST_TEXT),('bert_enc',TEST_TEXT_BERT)],
#                 'label':('label',TEST_LABEL)
#                 })

# TEST_TEXT.build_vocab(test_dataset, vectors=GloVe(name='840B', dim=300, cache='.embeddings'))
# outputs_cove_with_glove_test = MTLSTM(n_vocab=len(TEST_TEXT.vocab), vectors=TEST_TEXT.vocab.vectors, residual_embeddings=True, model_cache='.embeddings')

# test_iter = data.Iterator(
#     (test_dataset),
#     batch_size=5)

# print("Created Test Iterator")

# for batch_num, batch in enumerate(train_iter):
#     avg_loss = 0.0
#     glove_then_last_layer_cove = outputs_cove_with_glove(*batch.sentence)
#     target = torch.zeros(5, 512, 900)
#     max_sentence_len_in_batch = max(batch.sentence[1].tolist())
#     target[:, :max_sentence_len_in_batch, :] = glove_then_last_layer_cove
#     glove_then_last_layer_cove = target.permute(0,2,1)

#     bert_enc = batch.bert_enc.to(device)
#     glove_cove_enc = glove_then_last_layer_cove.to(device)

#     optimizer.zero_grad()
#     outputs = model(bert_enc, glove_cove_enc)
#     print(outputs.shape)
#     target = batch.label.reshape(5,1)
#     print(target.shape)

#     print(outputs, target)
#     loss = criterion(outputs, batch.label.long())
#     print(loss)
#     loss.backward()
#     optimizer.step()
#     print("Optimizer Complete")

#     avg_loss += loss.item()
#     print(avg_loss)

#     if batch_num % 50 == 49:
#         print('Fold: {}\t Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(foldId+1, epoch+1, batch_num+1, avg_loss/50))
#         avg_loss = 0.0

#     torch.cuda.empty_cache()
#     del bert_enc
#     del glove_cove_enc

#     break


# models = []
# temp_model = SuggestionClassifier()
# temp_model.load_state_dict(torch.load('jessi_A_fold_1_epoch_1.pt'))
# models.append(temp_model)
# temp_model = SuggestionClassifier()
# temp_model.load_state_dict(torch.load('jessi_A_fold_2_epoch_1.pt'))
# models.append(temp_model)